In [1]:

### Eficiencia computacional de nuestros algoritmos mediante la distancia de Kolmogorov

## Clases S4

#Crearemos la clase Kolmogorov-Binomial como prueba

#Clase Kolmogorov-Binomial con parametros n, p, potencia
setClass(
  "Kolmogorov-Binomial", representation(n="numeric", p="numeric", potencia="numeric"),
  prototype(n=NA_integer_, p=NA_real_, potencia= NA_integer_)
)


In [2]:
## Creamos los metodos de la clase S4

#Crearemos el metodo generico y un metodo para nuestra clase Kolmogorov-Binomial

#Metodo generico para el metodo kolmogorov
setGeneric(
  "kolmogorov",
  function(object){
    standardGeneric("kolmogorov")
  }
)

#Metodo kolmogorov sobre la distribucion Binomial
setMethod(
  "kolmogorov",
  signature("Kolmogorov-Binomial"),
  function(object){
    library("distrEx")
    distroptions(TruncQuantile = 1e-15)
    B1 <- Binom(size = object@n, prob = object@p)
    B2 <- convpow(B1, N = object@potencia)
    D1 <- as(B1, "LatticeDistribution")
    D2 <- convpow(D1, N = object@potencia)
    D3 <- KolmogorovDist(B2, D2)
  }
  
)


[1] "kolmogorov"

[1] "kolmogorov"

In [3]:
## Creamos nuestros objetos de prueba

#Crearemos 2 objetos de prueba para encontrar la eficacia de FFT sobre nuestra distribucion binomial

ejem6 <- new("Kolmogorov-Binomial",
          n=30, p= 0.8,                  # Parametros de distribucion binomial
          potencia= 10
)

ejem7 <- new("Kolmogorov-Binomial",
             n=50, p= 0.43,             # Parametros de distribucion binomial
             potencia= 18
)


In [4]:
### Ahora veamos la eficacia del metodo FFT, mediante la distancia de Kolmogorov


ejemplo2.conv <- kolmogorov(ejem6)
ejemplo2.conv

ejemplo3.conv <- kolmogorov(ejem7)
ejemplo3.conv


Loading required package: distr
Loading required package: startupmsg
Utilities for Start-Up Messages (version 0.9.4)
For more information see ?"startupmsg", NEWS("startupmsg")

Loading required package: sfsmisc
Loading required package: SweaveListingUtils
Utilities for Sweave Together with TeX 'listings' Package (version 0.7.7)
NOTE: Support for this package will stop soon.
Package 'knitr' is providing the same functionality in a better way.
Some functions from package 'base' are intentionally masked ---see SweaveListingMASK().
Note that global options are controlled by SweaveListingoptions() ---c.f. ?"SweaveListingoptions".
For more information see ?"SweaveListingUtils", NEWS("SweaveListingUtils")
There is a vignette to this package; try vignette("ExampleSweaveListingUtils").


Attaching package: ‘SweaveListingUtils’

The following objects are masked from ‘package:base’:

    library, require

Object Oriented Implementation of Distributions (version 2.6.2)
Attention: Arithmetics on di

Kolmogorov distance 
       2.319792e-06

Kolmogorov distance 
       3.147329e-06

In [5]:
### Ahora, como ultimo ejemplo, vamos a mostrar como hay varios pasos para obtener la convolucion 
###de la suma de distribuciones no centrales chi-squared

## Tenemos que tener en cuenta ciertos parametros

## df = grado de libertad
## ncp = parametro no centralizado 

#Primer metodo

library("distr")
distroptions(withgaps = FALSE)
distroptions(TruncQuantile = 1e-8)              # Parametro de truncamiento de FFT
distroptions(DefaultNrFFTGridPointsExponent = 18)

#Primer metodo (FFT1) para hallar chi-squared
df0 <- 4                  # Grado de libertad = 4
ncp0 <- 4                 #parametro no centralizado 
x0 <- 1.765               # x = 1.765
Z <- Norm(mean = sqrt(ncp0/df0))       # media igual a sqrt(ncp0/df0), varianza 1
Z2 <- Z^2                                 # Z2 <- Z^2  (chi-squared)
res1 <- convpow(Z2, N = df0)            # Convolucion de Z2  (Z2^4)



In [6]:


#Segundo metodo (FFT2)
Z <- Norm()             #Media igual a 0 , varianza 1
Z2 <- Z^2               # Z^2 (chi-squared)
X2 <- convpow(Z2, N = df0-1)           #Convolucion de de Z2^df0-1
Y2 <- Norm(mean = sqrt(ncp0))^2        #Y2 sera el ultimo factor a sumar 
res2 <- X2 + Y2               #Completamos la convolucion con el ultimo factor (Y2)


In [7]:
#Tercer metodo (FFT3)
res3 <- Chisq(df = df0-1) + Y2    # Tenemos la distribucion chi-squared(df0-1) + Y2(ultimo factor)


In [8]:
### Mostremos los resultados por medio del 1 metodo (FFT1), segundo metodo (FFT2), tercer metodo (FFT3)
###y el resultado obtenido por el comando directo Chi-squared

#Mostramos los 3 resultados obtenidos , de la siguient manera
res <- c(p(res1)(x0), p(res2)(x0), p(res3)(x0),
            +           pchisq(x0, df = df0, ncp = ncp0))
names(res) <- c("FFT1", "FFT2", "FFT3", "R")
res


FFT1       FFT2       FFT3          R 
0.04999865 0.04999924 0.04999936 0.04999937